# Part 1: E2E Example training object detection model using NVIDIA Pytorch Container from NGC
 ----

Note this Object Detection demo is based on https://github.com/pytorch/vision/tree/v0.11.3

This notebook walks you each step to train a model using containers from the NGC Catalog. We chose the GPU optimized Pytorch container as an example. The basics of working with docker containers apply to all NGC containers.

We will show you how to:

* Install the Docker Engine on your system
* Pull a Pytorch container from the NGC Catalog using Docker
* Run the Pytorch container using Docker
* Part 2 : Preprocess the Satellite imagery object detection dataset using the SAHI library
* Part 3 : Execute training a object detection on satellite imagery using TensorFlow and Jupyter Notebook
* Part 4 : Run inference on a trained object detection model using the SAHI library

Let's get started!

---

### 1. Install the Docker Engine
Go to https://docs.docker.com/engine/install/ to install the Docker Engine on your system.


### 2. Download the TensorFlow container from the NGC Catalog 

Once the Docker Engine is installed on your machine, visit https://ngc.nvidia.com/catalog/containers and search for the TensorFlow container. Click on the TensorFlow card and copy the pull command.
UPDATE IMG
<img src="https://raw.githubusercontent.com/kbojo/images/master/NGC.png">

Open the command line of your machine and past the pull command into your command line. Execute the command to download the container. 

```$ docker pull nvcr.io/nvidia/pytorch:21.11-py3```
    
The container starts downloading to your computer. A container image consists of many layers; all of them need to be pulled. 

### 3. Run the TensorFlow container image

Once the container download is completed, run the following code in your command line to run and start the container:

```$ docker run -it --gpus all  -p 8888:8888 -v $PWD:/projects --network=host nvcr.io/nvidia/pytorch:21.11-py3```
UPDATE IMG
<img src="https://raw.githubusercontent.com/kbojo/images/master/commandline1.png">

### 4. Install Jupyter lab and open a notebook

Within the container, run the following commands:

```pip install torchvision==0.11.3 jupyterlab```

```jupyter lab --ip=0.0.0.0 --port=8888 --allow-root```

Open up your favorite browser and enter: http://localhost:8888/?token=*yourtoken*.
UPDATE IMG
<img src="https://raw.githubusercontent.com/kbojo/images/master/commandline2.png">

You should see the Jupyter Lab application. Click on the plus icon to launch a new Python 3 noteb
ook.

Follow along with the image classification with the TensorFlow example provided below.

In [3]:
!pip install cython pycocotools matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 44.1 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0.6-cp38-cp38-linux_x86_64.whl size=425644 sha256=ede479a985b076b8d128e2b0704aeac62ebbb4cb010cccacf5b39b4e98db6d68
  Stored in directory: /tmp/pip-ephem-wheel-cache-o_xoxxl7/wheels/07/42/0a/26d14b3a7343223fcee09c101e73d82fff8bdd1541d85fe033
Successfully built pycocotools


# TLDR; run training job on 8 GPUS
The below cell will run a multi-gpu training job. This job will train an object detection model (faster-rcnn) on a dataset of satellite imagery images that contain 61 classes of objects
* Change `nproc_per_node` argument to specify the number of GPUs available on your server

In [ ]:
!torchrun --nproc_per_node=8 detection/train.py\
    --dataset coco --data-path=/run/determined/workdir/shared_fs/data/xview_dataset/ --model fasterrcnn_resnet50_fpn --epochs 26\
    --lr-steps 16 22 --aspect-ratio-group-factor 3

### 5. Preprocess Satellite Imagery Dataset with Pytorch
* TODO

In [1]:
# TODO

### 6. Object Detection on Satellite Imagery with Pytorch (Single GPU)
Follow and Run the code to train a Faster RCNN FPN (Resnet50 backbone) that classifies images of clothing. 

In [1]:
import sys
sys.path.insert(0,'detection')

In [2]:
# Import python dependencies
import datetime
import os
import time

import torch
import torch.utils.data
import torchvision
import torchvision.models.detection
import torchvision.models.detection.mask_rcnn

from coco_utils import get_coco, get_coco_kp

from group_by_aspect_ratio import GroupedBatchSampler, create_aspect_ratio_groups
from engine import train_one_epoch, evaluate

import presets
import utils
from train import *

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
output_dir='.'
data_path='/run/determined/workdir/shared_fs/data/xview_dataset/'
dataset_name='coco'
model='fasterrcnn_resnet50_fpn'
device='cuda'
batch_size=16
epochs=26
workers=4
lr=0.02
momentum=0.9
weight_decay=1e-4
lr_scheduler='multisteplr'
lr_step_size=8
lr_steps=[16, 22]
lr_gamma=0.1
print_freq=20
resume=False
start_epoch=0
aspect_ratio_group_factor=3
rpn_score_thresh=None
trainable_backbone_layers=None
data_augmentation='hflip'
pretrained=True
test_only=False
sync_bn=False

In [8]:
# if output_dir:
#         utils.mkdir(output_dir)
device = torch.device(device)
print(device)

cuda


In [9]:
# Import the dataset.
# Data loading code
print("Loading data")

dataset, num_classes = get_dataset(dataset_name, "train", get_transform(True, data_augmentation),
                                   data_path)
dataset_test, _ = get_dataset(dataset_name, "val", get_transform(False, data_augmentation), data_path)

print("Creating data loaders")
train_sampler = torch.utils.data.RandomSampler(dataset)
test_sampler = torch.utils.data.SequentialSampler(dataset_test)
group_ids = create_aspect_ratio_groups(dataset, k=aspect_ratio_group_factor)
train_batch_sampler = GroupedBatchSampler(train_sampler, group_ids, batch_size)
train_batch_sampler = torch.utils.data.BatchSampler(
            train_sampler, batch_size, drop_last=True)

data_loader = torch.utils.data.DataLoader(
    dataset, batch_sampler=train_batch_sampler, num_workers=workers,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1,
    sampler=test_sampler, num_workers=workers,
    collate_fn=utils.collate_fn)

Loading data
loading annotations into memory...
Done (t=3.59s)
creating index...
index created!
self.catIdtoCls:  {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 12, 12: 13, 13: 14, 14: 15, 15: 16, 16: 17, 17: 18, 18: 19, 19: 20, 20: 21, 21: 22, 22: 23, 23: 24, 24: 25, 25: 26, 26: 27, 27: 28, 28: 29, 29: 30, 30: 31, 31: 32, 32: 33, 33: 34, 34: 35, 35: 36, 36: 37, 37: 38, 38: 39, 39: 40, 40: 41, 41: 42, 42: 43, 43: 44, 44: 45, 45: 46, 46: 47, 47: 48, 48: 49, 49: 50, 50: 51, 51: 52, 52: 53, 53: 54, 54: 55, 55: 56, 56: 57, 57: 58, 58: 59, 59: 60}
self.num_classes:  61
{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 33: 32, 34: 33, 35: 34, 36: 35, 37: 36, 38: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44, 46: 45, 47: 46, 48: 47, 49: 48, 50: 49, 51: 50, 52: 51, 53: 

In [ ]:
# Let's have a look at one of the images. The following code visualizes the images using the matplotlib library.



In [ ]:
# Let's look again at the first ten images, but this time with the class names.



In [ ]:
# Let's build the model:



In [ ]:
# Compile the model:
# Define loss function, optimizer, and metrics.



In [ ]:
# Train the model:
# Let's train 1 epoch. After every epoch, training time, loss, and accuracy will be displayed.


In [ ]:
# Let's see how the model performs on the test data:
